# Get SPACE TRACK data

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
from datetime import timedelta
import numpy as np
import glob
import json

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### API config

In [4]:
# username = 'mstiles@grid.news'
# password = 'thespacerace123'
# output = 'json'

In [5]:
!curl -c 'params/cookies.txt' -b 'params/cookies.txt' 'https://www.space-track.org/ajaxauth/login' -d 'identity=mstiles@grid.news&password=thespacerace123'

""

### Boxscore

In [6]:
!curl --limit-rate 100K --cookie 'params/cookies.txt' 'https://www.space-track.org/basicspacedata/query/class/boxscore' -o 'data/raw/boxscore.json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35078  100 35078    0     0   105k      0 --:--:-- --:--:-- --:--:--  108k


In [7]:
boxscore = pd.read_json("data/raw/boxscore.json")

In [8]:
boxscore.columns = boxscore.columns.str.lower()

In [9]:
boxscore = boxscore[boxscore["country"] != "ALL"].sort_values(
    "decayed_total_count", ascending=False
)

In [10]:
boxscore.country = (
    boxscore.country.str.title()
    .str.replace("Of", "of", regex=False)
    .str.replace("Peoples", "People's", regex=False)
)

In [11]:
boxscore.iloc[1]

country                      United States of America
spadoc_cd                                          US
orbital_tba                                        16
orbital_payload_count                            4012
orbital_rocket_body_count                         725
orbital_debris_count                             4477
orbital_total_count                              9230
decayed_payload_count                            1370
decayed_rocket_body_count                         726
decayed_debris_count                             4757
decayed_total_count                              6853
country_total                                   16083
Name: 106, dtype: object

In [12]:
boxscore_slim = boxscore[
    [
        "country",
        "spadoc_cd",
        "orbital_tba",
        "orbital_payload_count",
        "orbital_rocket_body_count",
        "orbital_debris_count",
        "orbital_total_count",
    ]
].sort_values("orbital_debris_count", ascending=False)

boxscore_slim.head(20)

,country,spadoc_cd,orbital_tba,orbital_payload_count,orbital_rocket_body_count,orbital_debris_count,orbital_total_count
18,Commonwealth of Independent States,CIS,5,1552,1048,6043,8648
106,United States of America,US,16,4012,725,4477,9230
77,People's Republic of China,PRC,82,517,195,3656,4450
32,France,FR,0,80,165,355,600
40,India,IND,0,103,40,75,218
49,Japan,JPN,6,205,62,54,327
25,European Space Agency,ESA,0,96,7,53,156
16,People's Republic of China/Brazil,CHBZ,0,4,0,37,41
72,Orbital Telecommunications Satellite (Globalstar),ORB,0,58,0,17,75
28,European Organization For The Exploitation of Meteorological Satellites,EUME,0,9,0,8,17


In [13]:
boxscore["orbital_total_count"].sum()

25515

In [14]:
countries = boxscore[["country", "spadoc_cd"]]

In [15]:
countries.head()

,country,spadoc_cd
18,Commonwealth of Independent States,CIS
106,United States of America,US
77,People's Republic of China,PRC
32,France,FR
40,India,IND


In [16]:
boxscore.to_csv("data/processed/boxscore.csv", index=False)

In [17]:
!curl --limit-rate 100K --cookie 'params/cookies.txt'  'https://www.space-track.org/basicspacedata/query/class/satcat/format/json'  -O 'data/raw/all.json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23.0M  100 23.0M    0     0   100k      0  0:03:55  0:03:55 --:--:--  102k 0     0   100k      0  0:03:55  0:00:17  0:03:38   99k3:55  0:02:02  0:01:53  100k0:01:20   99k99k      0  0:03:56  0:03:20  0:00:36  100k6  0:03:32  0:00:24   99k  0    99k      0  0:03:56  0:03:45  0:00:11   99k
curl: (6) Could not resolve host: data


In [18]:
with open("data/raw/data.json", "r") as datafile:
    data = json.load(datafile)
all_df = pd.DataFrame(data)

In [19]:
all_df.head()

,INTLDES,NORAD_CAT_ID,OBJECT_TYPE,SATNAME,COUNTRY,LAUNCH,SITE,DECAY,PERIOD,INCLINATION,APOGEE,PERIGEE,COMMENT,COMMENTCODE,RCSVALUE,RCS_SIZE,FILE,LAUNCH_YEAR,LAUNCH_NUM,LAUNCH_PIECE,CURRENT,OBJECT_NAME,OBJECT_ID,OBJECT_NUMBER
0,1957-001A,1,ROCKET BODY,SL-1 R/B,CIS,1957-10-04,TTMTR,1957-12-01,96.19,65.10,938,214,None,4,0,LARGE,1,1957,1,A,Y,SL-1 R/B,1957-001A,1
1,1958-001A,4,PAYLOAD,EXPLORER 1,US,1958-02-01,AFETR,1970-03-31,88.48,33.15,215,183,None,None,0,None,1,1958,1,A,Y,EXPLORER 1,1958-001A,4
2,1958-003A,6,PAYLOAD,EXPLORER 3,US,1958-03-26,AFETR,1958-06-28,103.60,33.50,1739,117,None,5,0,None,1,1958,3,A,Y,EXPLORER 3,1958-003A,6
3,1958-004A,7,ROCKET BODY,SL-1 R/B,CIS,1958-05-15,TTMTR,1958-12-03,102.74,65.14,1571,206,None,None,0,None,1,1958,4,A,Y,SL-1 R/B,1958-004A,7
4,1958-004B,8,PAYLOAD,SPUTNIK 3,CIS,1958-05-15,TTMTR,1960-04-06,88.43,65.06,255,139,None,None,0,LARGE,1,1958,4,B,Y,SPUTNIK 3,1958-004B,8


In [20]:
all_df.columns = all_df.columns.str.lower()

In [21]:
len(all_df)

51584

In [22]:
all_df["norad_cat_id"] = all_df["norad_cat_id"].astype(str)

In [23]:
all_df.object_type.value_counts()

DEBRIS         33037
PAYLOAD        11974
ROCKET BODY     6305
TBA              268
Name: object_type, dtype: int64

In [24]:
all_df.rcs_size.value_counts()

SMALL     22351
LARGE     11731
MEDIUM     7430
Name: rcs_size, dtype: int64

In [25]:
active = all_df[all_df["decay"].isnull()]

In [26]:
len(active)

25515

In [27]:
active[active["norad_cat_id"].str.contains("12406")]

,intldes,norad_cat_id,object_type,satname,country,launch,site,decay,period,inclination,apogee,perigee,comment,commentcode,rcsvalue,rcs_size,file,launch_year,launch_num,launch_piece,current,object_name,object_id,object_number
34454,1978-014F,12406,DEBRIS,KYOKKO 1 DEB,JPN,1978-02-04,KSCUT,None,132.10,65.89,3780,647,None,None,0,SMALL,7966,1978,14,F,Y,KYOKKO 1 DEB,1978-014F,12406


In [28]:
active.to_csv("data/processed/space_track_active.csv", index=False)

In [29]:
active_grouped = (
    active.groupby(["launch_year", "object_type"])
    .agg({"norad_cat_id": "launch_year"})
    .reset_index()
).rename(columns={"norad_cat_id": "count"})

AttributeError: 'SeriesGroupBy' object has no attribute 'launch_year'

In [ ]:
alt.Chart(active_grouped).mark_bar().encode(
    x="launch_year", y="count", color="object_type"
).properties(width=650, height=350)

In [ ]:
active_grouped_cumsum = (
    active_grouped.groupby(["launch_year", "object_type"])
    .sum()
    .groupby("object_type")
    .cumsum()
    .reset_index()
    .rename(columns={"count": "cumsum"})
)

In [ ]:
active_grouped_cumsum

In [ ]:
alt.Chart(active_grouped_cumsum).mark_bar().encode(
    x="launch_year", y="cumsum", color="object_type"
).properties(width=650, height=350)

In [ ]:
alt.Chart(active_grouped_cumsum).mark_line().encode(
    x="launch_year:O",
    y="cumsum",
    color="object_type",
).properties(width=500, height=300)

In [ ]:
active_grouped_cumsum.to_csv(
    "data/processed/active_debris_categories_cumsum.csv", index=False
)

In [ ]:
alt.Chart(active_grouped).mark_bar().encode(
    x="launch_year:O",
    y="count",
    color=alt.Color("object_type"),
).properties(width=800, height=500)

In [ ]:
len(active[(active["launch_year"] == "1999") & (active["object_type"] == "DEBRIS")])

In [ ]:
active[
    (active["launch_year"] == "1999")
    & (active["object_type"] == "DEBRIS")
]